In [1]:
!python -m pip install langchain-community
#First cell - imports
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms.base import LLM
from typing import Optional, List
from pydantic import Field
import gradio as gr
import openai

print("✅ Imports successful!")

# Second cell - load documents
print("Loading documents...")
loader = DirectoryLoader('data', glob='**/*.txt')
documents = loader.load()
print(f"✅ Loaded {len(documents)} documents")

# Third cell - create embeddings
print("Generating embeddings...")
embedding_model = OllamaEmbeddings(model='llama3.2')
vectordb = Chroma.from_documents(documents, embedding=embedding_model, persist_directory="./chroma_db")
vectordb.persist()
print("✅ Embeddings created and stored")


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
✅ Imports successful!
Loading documents...


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


✅ Loaded 1 documents
Generating embeddings...


/var/folders/03/7rb54f0953bfshpgnhk88_sh0000gn/T/ipykernel_24341/342779084.py:23: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model='llama3.2')


✅ Embeddings created and stored


/var/folders/03/7rb54f0953bfshpgnhk88_sh0000gn/T/ipykernel_24341/342779084.py:25: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [ ]:
# Step 1: Load documents from the "data" directory
# Load documents from the "data" directory (TXT and PDF files)
print("Loading documents...")
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

# Load text files
txt_loader = DirectoryLoader('data', glob='**/*.txt')
txt_documents = txt_loader.load()

# Load PDF files
pdf_loader = DirectoryLoader('data', glob='**/*.pdf', loader_cls=PyPDFLoader)
pdf_documents = pdf_loader.load()

# Combine all documents
documents = txt_documents + pdf_documents
print(f"✅ Loaded {len(txt_documents)} text files and {len(pdf_documents)} PDF files")
print(f"✅ Total documents: {len(documents)}")

In [2]:

# Step 2: Create embeddings and store them in ChromaDB
print("Generating embeddings...")
embedding_model = OllamaEmbeddings(model='llama3.2')
vectordb = Chroma.from_documents(documents, embedding=embedding_model, persist_directory="./chroma_db")
vectordb.persist()

Generating embeddings...


In [3]:

# Step 3: Define custom LLM wrapper for llama3.2 using completion endpoint
class SimpleOllamaLLM(LLM):
    model: str = Field(default="llama3.2", description="Model name to use.")
    base_url: str = Field(default="http://localhost:11434/v1", description="Base URL for Ollama API.")
    api_key: str = Field(default="ollama", description="Dummy API key for compatibility.")

    def _call(self, prompt: str, stop: Optional[List[str]] = None, run_manager=None) -> str:
        client = openai.OpenAI(base_url=self.base_url, api_key=self.api_key)
        response = client.completions.create(
            model=self.model,
            prompt=prompt,
            stop=stop,
        )
        return response.choices[0].text.strip()

    @property
    def _llm_type(self) -> str:
        return "simple_ollama"

In [4]:

# Step 4: Initialize LLM and QA pipeline
llm = SimpleOllamaLLM(model="llama3.2")
retriever = vectordb.as_retriever()
qa_pipeline = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [5]:
# Step 5: Define chatbot logic with toggle
def chatbot(query, use_personal_data):
    if use_personal_data:
        return qa_pipeline.run(query)
    else:
        return llm._call(query)

# Embed logo (you can also use your local image path or base64 string)
logo_path = "D:/study/Projects/image.png"  # Your local image path

with gr.Blocks(title="Smart AI Chatbot", css="""
    #main-col {
        max-width: 600px;
        margin: 0 auto !important;
        padding-top: 20px;
    }
    .gr-button {
        max-width: 200px;
        margin: 0 auto;
    }
    .gr-textbox textarea {
        font-size: 14px;
    }
    .center-image img {
        display: block;
        margin-left: auto;
        margin-right: auto;
    }
""") as demo:

    with gr.Column(elem_id="main-col"):
        gr.Image(value=logo_path, width=120, show_label=False, container=False, elem_classes=["center-image"])

        gr.Markdown("## 🤖 Smart AI Chatbot", elem_id="title")
        gr.Markdown("Ask anything — toggle between general knowledge and your own data!", elem_id="subtitle")

        chatbot_ui = gr.Chatbot(label="💬 Chat History")
        input_box = gr.Textbox(label="💬 Your Question", placeholder="Type your message here...", lines=1)
        use_personal = gr.Checkbox(label="📚 Use Personal Documents?")
        btn = gr.Button("🚀 Submit", size="sm")
        state = gr.State([])

        def chat_handler(user_input, use_personal, chat_history):
            if use_personal:
                bot_reply = qa_pipeline.run(user_input)
            else:
                bot_reply = llm._call(user_input)

            chat_history.append((user_input, bot_reply))
            return chat_history, chat_history

        btn.click(fn=chat_handler, inputs=[input_box, use_personal, state], outputs=[chatbot_ui, state])

demo.launch()

/var/folders/03/7rb54f0953bfshpgnhk88_sh0000gn/T/ipykernel_24341/1899305341.py:37: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(label="💬 Chat History")


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


/var/folders/03/7rb54f0953bfshpgnhk88_sh0000gn/T/ipykernel_24341/1899305341.py:45: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  bot_reply = qa_pipeline.run(user_input)
